In [35]:
## Import Everythin ##

import keras
import pickle
import pandas as pd
from keras.models import Sequential
#from keras.metrics import 
import keras
import keras.preprocessing.text
from __future__ import absolute_import
from __future__ import division

import xgboost
from xgboost import XGBRegressor



import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score

import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesRegressor

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM

import string
import sys
import numpy as np
import pandas as pd
from six.moves import range
from six.moves import zip
import matplotlib.pyplot as plt
import warnings

from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from sklearn import svm

from sklearn import tree

from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor

In [2]:
#Prep the data

file = open("data/noConst_and_OHE.p","rb")
originaldata = pd.read_csv('data/train.csv', index_col='ID')
testdata = pd.read_csv('data/test.csv',index_col='ID')

###########

print(originaldata.iloc[[1]])

########

train = pd.read_csv('data/train.csv', index_col='ID')
y = train['y']
train.drop('y', axis=1, inplace=True)
test = pd.read_csv('data/test.csv', index_col='ID')
constcols_train = list(train.loc[:,train.apply(lambda i: len(i.unique()) == 1)].columns)
goodcols = list(set(train.columns).intersection(set(test.columns)) - set(constcols_train))
train = train[goodcols]
test = test[goodcols]
tot = pd.concat([train, test]).sort_index()
enumcols = list(tot.select_dtypes(include=['object']).columns)
bincols = list(tot.select_dtypes(include=['int64']).columns)

totOHE = pd.get_dummies(tot, columns=enumcols)
trainOHE = totOHE.loc[train.index]

testOHE = totOHE.loc[test.index]

        y X0 X1  X2 X3 X4 X5 X6 X8  X10  ...   X375  X376  X377  X378  X379  \
ID                                       ...                                  
6   88.53  k  t  av  e  d  y  l  o    0  ...      1     0     0     0     0   

    X380  X382  X383  X384  X385  
ID                                
6      0     0     0     0     0  

[1 rows x 377 columns]


In [3]:


#####

matarr  = trainOHE.as_matrix()
testarr = testOHE.as_matrix()

#####

Input_Length = len(trainOHE.T) 

#####

X_train1 = matarr[:3000,:]
Y_train1 = y[:3000]
X_train2 = matarr[3000:,:]
Y_train2 = y[3000:]
x = matarr
y = y

In [4]:
print(testOHE.shape)
print(trainOHE.shape)

(4209, 567)
(4209, 567)


In [5]:
print(testOHE)

      X71  X59  X363  X34  X117  X369  X257  X277  X92  X300  ...   X1_p  \
ID                                                            ...          
1       0    0     1    0     0     0     0     0    0     0  ...      0   
2       0    0     0    0     0     0     0     0    0     0  ...      0   
3       0    0     1    0     0     0     0     0    0     0  ...      0   
4       0    0     1    0     0     0     0     0    0     0  ...      0   
5       0    0     1    0     0     0     0     0    0     1  ...      0   
8       0    0     0    0     0     0     0     0    0     0  ...      0   
10      0    0     1    0     0     0     0     0    0     0  ...      0   
11      0    0     1    0     0     0     0     0    0     0  ...      0   
12      0    0     1    0     0     0     0     0    0     0  ...      0   
14      0    0     1    0     0     0     0     0    0     0  ...      0   
15      0    0     1    0     0     0     0     0    0     0  ...      0   
16      0   

In [6]:
print(trainOHE)

      X71  X59  X363  X34  X117  X369  X257  X277  X92  X300  ...   X1_p  \
ID                                                            ...          
0       0    0     0    0     0     0     0     0    0     0  ...      0   
6       0    0     1    0     0     0     0     0    0     0  ...      0   
7       1    0     1    0     0     0     0     0    0     0  ...      0   
9       1    0     1    0     0     0     0     0    0     0  ...      0   
13      0    0     1    0     0     0     0     0    0     0  ...      0   
18      0    0     0    0     0     0     0     0    0     0  ...      0   
24      0    0     1    0     0     0     0     0    0     0  ...      0   
25      0    0     1    0     0     0     0     0    0     1  ...      0   
27      0    0     1    0     0     0     0     0    0     1  ...      0   
30      0    0     1    0     0     0     0     0    0     0  ...      0   
31      0    0     0    0     1     0     0     0    0     0  ...      0   
32      0   

In [7]:
matarr.shape

(4209, 567)

In [8]:
print(Input_Length)

567


In [9]:
print(X_train1.shape)
print(X_train2.shape)


(3000, 567)
(1209, 567)


In [10]:
#basic Neural Net for Regression

modelbasic = Sequential()
modelbasic.add(Dense(15,input_dim=Input_Length,kernel_initializer='normal', activation='relu'))
modelbasic.add(Dense(30, kernel_initializer='normal', activation='tanh'))
#modelbasic.add(Dropout(0.2))
modelbasic.add(Dense(20, kernel_initializer='normal', activation='relu'))
modelbasic.add(Dense(1))


modelbasic.compile(loss='mean_squared_error', optimizer='adam')

In [251]:
#function model

def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(13, input_dim=13, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal',dropout=0.2))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [288]:
runJob = modelbasic.fit(x, y, batch_size=1, epochs=15, verbose=1, shuffle=False)

Epoch 1/15
4209/4209 [==============================] - 3s - loss: 59.3967     
Epoch 2/15
4209/4209 [==============================] - 3s - loss: 58.9482     
Epoch 3/15
4209/4209 [==============================] - 3s - loss: 57.2151     
Epoch 4/15
4209/4209 [==============================] - 3s - loss: 57.8172     
Epoch 5/15
4209/4209 [==============================] - 3s - loss: 55.6410     
Epoch 6/15
4209/4209 [==============================] - 3s - loss: 54.7940     
Epoch 7/15
4209/4209 [==============================] - 3s - loss: 53.0056     
Epoch 8/15
4209/4209 [==============================] - 3s - loss: 52.1385     
Epoch 9/15
4209/4209 [==============================] - 3s - loss: 50.9219     
Epoch 10/15
4209/4209 [==============================] - 3s - loss: 50.1070     
Epoch 11/15
4209/4209 [==============================] - 3s - loss: 48.8866     
Epoch 12/15
4209/4209 [==============================] - 3s - loss: 48.4482     
Epoch 13/15
4209/4209 [==============

In [23]:
seed = 7
numpy.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=100, batch_size=5, verbose=0)

In [11]:
# Random Forest

forestry = RandomForestRegressor(n_estimators=140,max_features='auto',max_depth=4,n_jobs=6,min_samples_split=270,min_samples_leaf=4,random_state=1,warm_start=True)
forestry.fit(X_train1,Y_train1)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=4,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=4,
           min_samples_split=270, min_weight_fraction_leaf=0.0,
           n_estimators=140, n_jobs=6, oob_score=False, random_state=1,
           verbose=0, warm_start=True)

In [12]:
# Extra Trees

extraX = ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_split=1e-07, min_samples_leaf=1,
          min_samples_split=2, min_weight_fraction_leaf=0.0,
          n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
          verbose=0, warm_start=False)
extraX.fit(X_train1,Y_train1)

ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_split=1e-07, min_samples_leaf=1,
          min_samples_split=2, min_weight_fraction_leaf=0.0,
          n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
          verbose=0, warm_start=False)

In [13]:
# XG Boost

xgmodel = XGBRegressor(base_score=0.2, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=110, nthread=-1,
       objective='reg:gamma', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)
xgmodel.fit(X_train1,Y_train1)

print("XG BOOST: ",r2_score(Y_train2, xgmodel.predict(X_train2)))

XG BOOST:  0.632918418386


In [14]:
#Gradient Boosting

clf = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,max_depth=4, random_state=0, loss='lad').fit(x,y)



In [15]:
# Support Vector

vectormach = svm.SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
vectormach.fit(X_train1, Y_train1) 

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [16]:
#Stochastic Gradient Descent



sgd = SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', n_iter=5, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, verbose=0, warm_start=False)
sgd.fit(x,y)

SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', n_iter=5, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, verbose=0, warm_start=False)

In [17]:
# Neighbors


neigh = KNeighborsRegressor(n_neighbors=40)
neigh.fit(X_train1, Y_train1)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=40, p=2,
          weights='uniform')

In [21]:
pd.DataFrame(runJob.history).plot(secondary_y='lr', figsize=(15,5), logy=True, grid=False);
plt.show();

NameError: name 'runJob' is not defined

In [18]:


print("R^2 Values\n")
print("Neural Network: ",r2_score(Y_train2, modelbasic.predict(X_train2)))
print("XG BOOST: ",r2_score(Y_train2, xgmodel.predict(X_train2)))
print("Stochastic Gradient: ",r2_score(Y_train2, sgd.predict(X_train2)))
print("Neighbors: ",r2_score(Y_train2, neigh.predict(X_train2)))
print("Support Vector Machine: ",r2_score(Y_train2, vectormach.predict(X_train2)))
print("Gradient Boosting: ",r2_score(Y_train2,clf.predict(X_train2)))
print("Random Forest: ",r2_score(Y_train2,forestry.predict(X_train2)))
print("Extra Trees Score: ", r2_score(Y_train2,extraX.predict(X_train2)))

R^2 Values
Neural Network:  -71.3569932917
XG BOOST:  0.632918418386
Stochastic Gradient:  0.63030552482
Neighbors:  0.489638793632
Support Vector Machine:  0.487945859128
Gradient Boosting:  0.659119699149
Random Forest:  0.620112424067
Extra Trees Score:  0.221659373315


In [82]:
#Save Gradient Boosting

pd.DataFrame(index=testOHE.index, data=clf.predict(testarr), columns=['y']).to_csv('submissionNickGradient6.csv')



In [289]:
# Save Neural Net

pd.DataFrame(index=testOHE.index, data=modelbasic.predict(testarr), columns=['y']).to_csv('submissionNickNeural6.csv')


In [245]:
# Save Random Forest

pd.DataFrame(index=testOHE.index, data=forestry.predict(testarr), columns=['y']).to_csv('Forestry9.csv')


In [29]:
# Save SGD

pd.DataFrame(index=testOHE.index, data=sgd.predict(testarr), columns=['y']).to_csv('sgd.csv')


In [51]:
# XG BOOST BABY

pd.DataFrame(index=testOHE.index, data=xgmodel.predict(testarr), columns=['y']).to_csv('xg3.csv')


In [14]:
# Below is Hyper-Parameter Training with GridSearchCV and other methods

In [ ]:
#Hyper Parameter 1 for Random Forest

number = [100,120,130,140,150,160,170,180,190,200,210,230,240,250,260,270,280,290,300]

for i in range(0,len(number)):
    forestry = RandomForestRegressor(n_estimators=number[i],max_features='auto',max_depth=9,n_jobs=2,min_samples_leaf=4 )
    forestry.fit(X_train1,Y_train1)
    print("Random Forest Score with ", number[i], "estimators: ", forestry.score(X_train2,Y_train2))

In [ ]:
# Hyper Parameter Test for Gradient Boosting



paramGB = {'n_estimators':list(range(100,150,10)),'max_depth':list(range(1,5))}


hyper = GradientBoostingRegressor()

hyperGB = GridSearchCV(hyper,paramGB)

hyperGB.fit(X_train1,Y_train1)

print(hyperGB)

In [ ]:
# R^2 Score from the Hyper Parameter Sets

print("Hyper Gradient Boosting: ",r2_score(Y_train2,hyperGB.predict(X_train2)))